In [ ]:
import json

file_path = '/home/jaesung/pulmuone/agent_2/test_result_1K.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
len(data)

1012

In [ ]:
data[1]

{'Input': ['조미김', ' 변성전분sa2', ' 마라소스', ' 밀가루'],
 'Similar Input': ['카놀라유',
  '참기름',
  '들기름',
  '김자반',
  '설탕',
  '볶음참깨',
  '칼슘소금',
  '계란난황립',
  '건조멸치',
  '고추씨기름',
  '건조새우'],
 'Response': [['content',
   "\r\n    너는 식품 개발자야. \r\n    출력 형식은 example들과 반드시 같도록 수정해줘.\r\n    \r\nQ: 원본 식재료 ['카놀라유', '참기름', '들기름', '김자반', '설탕', '볶음참깨', '칼슘소금', '계란난황립', '건조멸치', '고추씨기름', '건조새우']의 카테고리는 조미김이야. \r\n    ['변성전분sa2', '마라소스', '밀가루']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.\r\n\r\nA: 변성 전분 사2: 35%, 마라 소스: 20%, 밀가루: 15%, 참깨: 10%, 고춧가루: 10%, 건새우: 10%"],
  ['additional_kwargs', {}],
  ['response_metadata', {'finish_reason': 'stop'}],
  ['type', 'ai'],
  ['name', None],
  ['id', 'run-556f5aef-8936-4b73-a45b-83d1370bb1dc-0'],
  ['example', False],
  ['tool_calls', []],
  ['invalid_tool_calls', []],
  ['usage_metadata', None]]}

In [5]:
import re
import pandas as pd

df_list = []

for i in range(len(data)):

    Q = data[i]['Input']

    if not data[i]['Response'] or data[i]['Response'] == None:
        pass
    else:
        A_origin = data[i]['Response'][0][1].split('A:')[-1].strip()

        # '배합비' 부분을 추출하는 정규 표현식 (재료명과 % 형식)
        matches = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%', A_origin)
        # matches = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%|[가-힣a-zA-Z0-9]+\s*\d+%', A_origin)

        # 추출된 항목을 콤마로 연결하여 출력
        A = ', '.join(matches)

        # print(Q)
        # print('')
        # print(A)
        # print('')

        len_ox = 0
        for j in range(1, 4):
            item = Q[j].strip()
            
            if item in A:
                len_ox += 1

        # print(len_ox)        

        # 정규 표현식을 사용해 ':' 뒤에 있는 숫자들을 추출 (숫자 + % 형식)
        numbers = re.findall(r':\s*(\d+)%', A)

        # 추출된 숫자들을 정수로 변환 후 합산
        total_sum = sum(map(int, numbers))

        # print('')
        # print(f"숫자의 합: {total_sum}")
        
        # 데이터프레임에 추가할 항목 생성
        df_list.append({
            'Q': Q,
            'A': A,
            'len_ox': len_ox,
            'total_sum': total_sum
        })
    
df = pd.DataFrame(df_list)

In [6]:
len(df)

886

In [7]:
df.head()

,Q,A,len_ox,total_sum
0,"[강화우유, 딸기퓨레, 건면용소맥분, 소맥전분]","딸기퓨레: 35%, 건면용소맥분: 20%, 소맥전분: 15%, 비타민a혼합제제: 1...",3,100
1,"[조미김, 변성전분sa2, 마라소스, 밀가루]","사2: 35%, 소스: 20%, 밀가루: 15%, 참깨: 10%, 고춧가루: 10%...",1,100
2,"[염지란, 대두유, 정제수, 5리보뉴클레오티드이나트륨]","대두유: 30%, 정제수: 25%, 5리보뉴클레오티드이나트륨: 15%, 간장: 10...",3,100
3,"[소스가열섭취냉동식품, 마늘, 고과당, 나타드코코]","마늘: 25%, 고과당: 30%, 나타드코코: 10%, 참기름: 10%, 양조간장:...",3,110
4,"[두부류또는묵류, 정제소금, 흑후추분말, 혼합야채엑기스]","유기농대두: 45%, 정제소금: 20%, 혼합야채엑기스: 15%, 흑후추분말: 10...",3,100


In [8]:
df[df['len_ox'] != 3]

,Q,A,len_ox,total_sum
1,"[조미김, 변성전분sa2, 마라소스, 밀가루]","사2: 35%, 소스: 20%, 밀가루: 15%, 참깨: 10%, 고춧가루: 10%...",1,100
5,"[냉면, 감자전분, 찹쌀, 멸치추출농축액]","전분: 35%, 찹쌀가루: 20%, 추출액: 15%, 소금: 10%, 제제: 10%...",1,100
6,"[발효유, 정제수, 비타민c, 양조간장]","정제수: 50%, C: 20%, 간장: 10%, 농축액: 10%, 농축액: 10%",1,100
7,"[건강기능식품신, 페퍼로니용분말, 정제소금, 치킨스톡p]","페퍼로니: 45%, 정제소금: 20%, 스톡: 15%, 혼합향료: 10%, D3: ...",1,100
8,"[국수, 설탕, 쌀단백분말, 정제소금]","설탕: 25%, 쌀가루: 30%, 밀가루: 20%, 소금: 10%, 고명: 10%,...",1,100
...,...,...,...,...
880,"[알가공품, 잔탄검, 난백, 정제소금]","정제수: 25%, 대두유: 15%, 난백: 10%, 정제소금: 10%, 치자황색소:...",2,110
881,"[식육함유가공품, 고춧가루, 백설탕, 흑후추분말]","밀가루: 35%, 고춧가루: 20%, 설탕: 10%, 후추분말: 10%, 소금: 1...",1,120
883,"[과채주스비가열, 양배추, 오월의된장, 난백분]","양배추: 35%, 된장: 20%, 난백분: 15%, 사과주스: 10%, 레몬즙: 1...",2,100
884,"[기타수산물가공품, 건조감자분말, 소프트마가린, 가람마살라분말2]","건조감자분말: 35%, 소프트마가린: 20%, 가람마살라분말: 15%, 정제소금: ...",2,110


In [9]:
df[df['total_sum'] != 100]

,Q,A,len_ox,total_sum
3,"[소스가열섭취냉동식품, 마늘, 고과당, 나타드코코]","마늘: 25%, 고과당: 30%, 나타드코코: 10%, 참기름: 10%, 양조간장:...",3,110
16,"[기타가공품가열섭취냉동식품, 정백당, 5리보뉴클레오티드이나트륨, 흑후추분말]","두부: 30%, 정제수: 25%, 밀가루: 15%, 양념: 10%, 설탕: 5%, ...",0,144
17,"[즉석조리식품살균, 두부, 복숭아농축액, 정제소금]","두부: 25%, 농축액: 15%, 대파: 10%, 청피망: 10%, 당근: 10%,...",2,90
19,"[가공유류, 당근즙, 우동지미인헨서, 파인애플즙]","당근즙: 25%, 우동지미인헨서: 20%, 파인애플즙: 15%, 코코아파우더: 10...",3,110
20,"[수산물, 비타민, 양파, 구연산삼나트륨]","C: 25%, 양파: 20%, 삼나트륨: 15%, 멸치: 10%, 천일염: 10%,...",1,90
...,...,...,...,...
878,"[즉석섭취식품, 과당, 나물콩, 냉동흰다리새우살]","백미: 35%, 찹쌀: 20%, 현미: 15%, 검은콩: 10%, 참깨: 5%, 양...",0,145
880,"[알가공품, 잔탄검, 난백, 정제소금]","정제수: 25%, 대두유: 15%, 난백: 10%, 정제소금: 10%, 치자황색소:...",2,110
881,"[식육함유가공품, 고춧가루, 백설탕, 흑후추분말]","밀가루: 35%, 고춧가루: 20%, 설탕: 10%, 후추분말: 10%, 소금: 1...",1,120
882,"[숙면주정처리, 당근, 정제수, 정제수]","당근: 25%, 정제수: 30%, 전분: 15%, 밀가루: 10%, 메밀가루: 10...",3,110


In [10]:
df[(df['len_ox'] != 3) | (df['total_sum'] != 100)]

,Q,A,len_ox,total_sum
1,"[조미김, 변성전분sa2, 마라소스, 밀가루]","사2: 35%, 소스: 20%, 밀가루: 15%, 참깨: 10%, 고춧가루: 10%...",1,100
3,"[소스가열섭취냉동식품, 마늘, 고과당, 나타드코코]","마늘: 25%, 고과당: 30%, 나타드코코: 10%, 참기름: 10%, 양조간장:...",3,110
5,"[냉면, 감자전분, 찹쌀, 멸치추출농축액]","전분: 35%, 찹쌀가루: 20%, 추출액: 15%, 소금: 10%, 제제: 10%...",1,100
6,"[발효유, 정제수, 비타민c, 양조간장]","정제수: 50%, C: 20%, 간장: 10%, 농축액: 10%, 농축액: 10%",1,100
7,"[건강기능식품신, 페퍼로니용분말, 정제소금, 치킨스톡p]","페퍼로니: 45%, 정제소금: 20%, 스톡: 15%, 혼합향료: 10%, D3: ...",1,100
...,...,...,...,...
881,"[식육함유가공품, 고춧가루, 백설탕, 흑후추분말]","밀가루: 35%, 고춧가루: 20%, 설탕: 10%, 후추분말: 10%, 소금: 1...",1,120
882,"[숙면주정처리, 당근, 정제수, 정제수]","당근: 25%, 정제수: 30%, 전분: 15%, 밀가루: 10%, 메밀가루: 10...",3,110
883,"[과채주스비가열, 양배추, 오월의된장, 난백분]","양배추: 35%, 된장: 20%, 난백분: 15%, 사과주스: 10%, 레몬즙: 1...",2,100
884,"[기타수산물가공품, 건조감자분말, 소프트마가린, 가람마살라분말2]","건조감자분말: 35%, 소프트마가린: 20%, 가람마살라분말: 15%, 정제소금: ...",2,110


In [11]:
df[(df['len_ox'] != 3) & (df['total_sum'] != 100)]

,Q,A,len_ox,total_sum
16,"[기타가공품가열섭취냉동식품, 정백당, 5리보뉴클레오티드이나트륨, 흑후추분말]","두부: 30%, 정제수: 25%, 밀가루: 15%, 양념: 10%, 설탕: 5%, ...",0,144
17,"[즉석조리식품살균, 두부, 복숭아농축액, 정제소금]","두부: 25%, 농축액: 15%, 대파: 10%, 청피망: 10%, 당근: 10%,...",2,90
20,"[수산물, 비타민, 양파, 구연산삼나트륨]","C: 25%, 양파: 20%, 삼나트륨: 15%, 멸치: 10%, 천일염: 10%,...",1,90
27,"[기타가공품, 정백당, 활성밀글루텐, 건조냉이]","돼지고기: 35%, 정제수: 20%, 양파: 10%, 밀가루: 10%, 건조냉이: ...",1,95
30,"[햄류, 전분가공품, 5리보뉴클레오티드나트륨, 우엉]","전분가공품: 30%, 우엉: 25%, 정백당: 15%, 간장: 10%, 소금: 10...",2,105
...,...,...,...,...
877,"[유산균음료살균, 물엿, 합성향료볶은콩향, 아미노베이스ps]","물엿: 35%, 합성향료볶은콩향: 20%, 아미노베이스PS: 10%, 자두농축액: ...",2,110
878,"[즉석섭취식품, 과당, 나물콩, 냉동흰다리새우살]","백미: 35%, 찹쌀: 20%, 현미: 15%, 검은콩: 10%, 참깨: 5%, 양...",0,145
880,"[알가공품, 잔탄검, 난백, 정제소금]","정제수: 25%, 대두유: 15%, 난백: 10%, 정제소금: 10%, 치자황색소:...",2,110
881,"[식육함유가공품, 고춧가루, 백설탕, 흑후추분말]","밀가루: 35%, 고춧가루: 20%, 설탕: 10%, 후추분말: 10%, 소금: 1...",1,120


In [14]:
df[df['len_ox'] ==0]

,Q,A,len_ox,total_sum
12,"[면류, 멸치추출농축액, 해바라기혼합유, 흑후추분말]","밀가루: 35%, 전분: 20%, 추출물: 10%, 해바라기유: 10%, 소금: 1...",0,100
16,"[기타가공품가열섭취냉동식품, 정백당, 5리보뉴클레오티드이나트륨, 흑후추분말]","두부: 30%, 정제수: 25%, 밀가루: 15%, 양념: 10%, 설탕: 5%, ...",0,144
22,"[냉면, 흑후추분말, 코코아리커dznm5020, 검은깨페이스트]","소맥분: 30%, 전분: 25%, 분말: 10%, 파우더: 10%, 가루: 5%, ...",0,100
38,"[과채주스비가열, 양조간장, 고소한빵가루, 라즈베리퓨레]","라즈베리: 45%, 사과주스: 20%, 레몬즙: 10%, 설탕: 10%, 정제수: ...",0,100
45,"[강화우유, 갈색설탕, 맛있는요리애, 멸치추출농축액]","설탕: 25%, 애호가: 20%, 농축액: 15%, 혼합제: 10%, B2: 10%...",0,100
...,...,...,...,...
858,"[과채음료비가열함유, 겨자분204, 유크림, 느타리버섯]","퀴노아: 35%, 케일: 20%, 바나나: 15%, 파프리카: 10%, 아보카도: ...",0,100
863,"[가공유저지방, 클로렐라분말, 콘스프맛베이스, 제삼인산칼슘]","우유: 45%, 탈지분유: 20%, 설탕: 10%, 농축액: 10%, 분말: 5%,...",0,100
870,"[소스류살균, 함수결정포도당, 아몬드슬라이스, 두부응고제]","함정결정포도당: 35%, 슬라이스: 20%, 응고제: 10%, 농축액: 15%, 레...",0,110
874,"[잼류, 사과농축과즙, 구기자농축액, 정제소금]","사과다이스: 45%, 농축액: 20%, 정백당: 15%, 레몬즙: 10%, 소금: 10%",0,100


In [13]:
print(df['A'][200])

양파: 35%, 설탕: 20%, 정제수: 15%, 소금: 10%, 레몬즙: 10%, 식용유: 10%
